This unit is to read the air temperature 
as well as log GPS (bluefly) and humidity to the SD card

We can get all the selected components working and producing 
same file format as the original one.  And then solder together

Then we need to design a 3D-printed housing for it that rigidly 
attaches to an upright profile

Then take it out and run around mapping warm and cold areas 

(Should work on a car ride)

* Fix the humidity work
* Fix the initial logger plot
* Fix the analog reader rate
* missing date records: Rt0000B90Fd"2017-06-24T13:44:54.600"e000F423Fn00000682f00000682o000F423FA


In [22]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [45]:
%sendtofile --source NewDataLogger_funcs.py
%sendtofile --quiet --source BlueFly_funcs.py


Sent 94 lines (3248 bytes) to NewDataLogger_funcs.py.
Sent 112 lines (4398 bytes) to BlueFly_funcs.py.


In [494]:
%sendtofile --binary --source ../Sensor_Kennel/OLED_driver.py
%sendtofile --mkdir --source /home/julian/extrepositories/micropython/drivers/sdcard/sdcard.py /lib/
%sendtofile --quiet --source ../Sensor_Kennel/SI7021_funcs.py
%sendtofile --quiet --source ../Sensor_Kennel/OLED_grapher.py
%sendtofile --quiet --source BlueFly_funcs.py
%sendtofile --quiet --source ../Sensor_Kennel/BNO055serial_funcs.py


Sent 55 lines (2040 bytes) to SI7021_funcs.py.


In [26]:
%sendtofile main.py

from OLED_driver import i2c, fbuff, oledshow, doublepixels, fatntext, oledshowfattext
from machine import Pin, SPI, UART, ADC
import sdcard, os, time
from OLED_grapher import scrollinit, addscrollgraph, plotscrollgraph
from BlueFly_funcs import uart, SetupGPS, BFVmakesettings, ParseNMEA
from NewDataLogger_funcs import ConnectSDcardFile, ConnectBluefly, readlogSI7021, calchumidtemps, readlogbluefly
     
# SD card uses sck=5, mosi=22, miso=27, cs=23
# OLED uses rst=16, scl=15, sda=4
# uartBlueFly = UART(1, baudrate=57600, rx=13, tx=17)
# Pin36 nickel input, Pin19 bridge resistor
# uartBNO055 = UART(2, baudrate=115200, rx=2, tx=18)

# PRG button used to cycle through modes
butt0 = Pin(0)  
butt0presses = 0
def butt0press(e):
    global butt0presses
    butt0presses += 1
butt0.irq(butt0press, butt0.IRQ_FALLING)

# input value from wire sensor
nickelwire = ADC(Pin(36))
nickelwire.atten(nickelwire.ATTN_11DB)
nickelwire.width(nickelwire.WIDTH_12BIT)
nickelbridge = Pin(19, Pin.OUT)
nickelbridge.value(0)

ConnectBluefly()
sdfile = ConnectSDcardFile()
time.sleep_ms(1000) # show file name

SI7021readfail = 0
try:
    i2c.writeto(0x40, b'\xFE')  # resets chip
except OSError:
    SI7021readfail = -1
    oledshowfattext(["SI7021", "bad"])
    time.sleep_ms(700)

    
prevflushstamp = 0
prevoledstamp = 0
prevnickelwireread = -100
prevnickelwirestamp = 0
scrollinit()
gpsfixes = 0
prevbutt0 = butt0.value()
prevdisplaymode = 0
prsrec = 100000
while True:
    tstamp = time.ticks_ms()
    displaymode = butt0presses%4
    
    # should use mag = Pin(25, Pin.OUT) to tune it, but doesn't work
    nickelwireread = nickelwire.read()
    if abs(nickelwireread - prevnickelwireread) >= 4 and tstamp > prevnickelwirestamp + 25:
        sdfile.write("Nt{:08X}s{:06X}\n".format(tstamp, nickelwireread))
        prevnickelwireread = nickelwireread
        prevnickelwirestamp = tstamp
        
    if SI7021readfail != -1:
        try:
            readlogSI7021()
        except OSError:
            SI7021readfail += 1
            
    lbf = readlogbluefly()
    
    if tstamp > prevflushstamp + 2000:
        sdfile.flush()
        prevflushstamp = tstamp

    # all data has been collected.  Now display it
    if type(lbf) == int:
        prsrec = lbf
                
    if displaymode != prevdisplaymode:
        scrollinit()
        oledshowfattext(["display%d"%displaymode])
        prevdisplaymode = displaymode
        
    if displaymode == 0:  # display nmea situation
        if type(lbf) == bytes:
            oledshowfattext([lbf[:8], lbf[8:16], lbf[16:24], lbf[24:32]])
            print(lbf)
                
    elif tstamp > prevoledstamp + 200: # scrolling values
        if displaymode == 1:
            addscrollgraph(prsrec-100000, tstamp)
            plotscrollgraph(fbuff)
            fbuff.text("prs",0,0,1)
            fbuff.text(str(prsrec),0,8,1)
        elif displaymode == 2:
            addscrollgraph(nickelwireread, tstamp)
            plotscrollgraph(fbuff)
            fbuff.text("nickel",0,0,1)
            fbuff.text(str(nickelwireread),0,8,1)
        elif displaymode == 3:
            hs, ts = calchumidtemps()
            addscrollgraph(ts, tstamp)
            plotscrollgraph(fbuff)
            fbuff.text("temp",0,0,1)
            fbuff.text(str(nickelwireread),0,8,1)
        else:
            oledshowfattext(["mstamp", str(tstamp)])
        oledshow()
        prevoledstamp = tstamp


No serial connected
  %serialconnect to connect
  %esptool to flash the device
  %lsmagic to list commands

In [25]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.9.3-603-gfb7dabb9 on 2018-05-02; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7fc8c61ab7b8, open=True>(port='/dev/ttyUSB1', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [53]:

%fetchfile --binary "sd/LOG/012.TXT"


Fetched 192004=192004 bytes from sd/LOG/012.TXT.
Saving file to '012.TXT'

In [1395]:
ConnectBluefly()


b'BVL' already good
b'BHV' already good
b'BOF' already good
b'BHT' already good
Change GPS to baud 57600


In [1398]:
for i in range(1000):
    x = uart.readline()
    if x[:3] != b'PRS':
        print(x)


b'$GPGGA,000007.800,,,,,0,0,,,M,,M,,*47\r\n'
b'$GPVTG,0.00,T,,M,0.00,N,0.00,K,N*32\r\n'
b'GGA,000045.200,,,,,0,0,,,M,,M,,*4B\r\n'
b'$GPVTG,0.00,T,,M,0.00,N,0.00,K,N*32\r\n'
b'4\r\n'
b'$GPGGA,000048.400,,,,,0,0,,,M,,M,,*40\r\n'
b'$GPVTG,0.00,T,,M,0.00,N,0.00,K,N*32\r\n'


Traceback (most recent call last):
  File "<stdin>", line 3, in <module>
TypeError: 'NoneType' object is not subscriptable
